## Clone OpenKE for benchmark datasets FB15K-237 and WN18

In [ ]:
from typing import Optional, Union, List, Callable, Dict
from tqdm import tqdm_notebook as tqdm
from functools import partial
from pathlib import Path
import pandas as  pd
import numpy as np
import traceback
import warnings
import logging
import random
import pickle
import wandb



# MyTorch imports
from mytorch.utils.goodies import *
from mytorch import dataiters

# Local imports 
from utils import *

## Prepare data

In [ ]:
# Overwriting data dir
RAW_DATA_DIR = Path('./OpenKE/benchmarks/FB15K237')
DATASET = 'fb15k'

np.random.seed(42)
random.seed(42)

In [ ]:
training_triples = []
valid_triples = []
test_triples = []

with open(RAW_DATA_DIR / "entity2id.txt", "r") as ent_file, \
    open(RAW_DATA_DIR / "relation2id.txt", "r") as rel_file, \
    open(RAW_DATA_DIR / "train2id.txt", "r") as train_file, \
    open(RAW_DATA_DIR / "valid2id.txt", "r") as valid_file, \
    open(RAW_DATA_DIR / "test2id.txt", "r") as test_file:
    num_entities = int(next(ent_file).strip("\n"))
    num_relations = int(next(rel_file).strip("\n"))
    num_trains = int(next(train_file).strip("\n"))
    for line in train_file:
        triple = line.strip("\n").split(" ")
        training_triples.append([int(triple[0]), int(triple[2]), int(triple[1])])
        
    num_valid = int(next(valid_file).strip("\n"))
    for line in valid_file:
        triple = line.strip("\n").split(" ")
        valid_triples.append([int(triple[0]), int(triple[2]), int(triple[1])])
    
    num_test = int(next(test_file).strip("\n"))
    for line in test_file:
        triple = line.strip("\n").split(" ")
        test_triples.append([int(triple[0]), int(triple[2]), int(triple[1])])

In [ ]:
EXPERIMENT_CONFIG = {
    'NUM_ENTITIES': num_entities,
    'NUM_RELATIONS': num_relations,
    'EMBEDDING_DIM': 200,
    'NORM_FOR_NORMALIZATION_OF_ENTITIES': 2,
    'NORM_FOR_NORMALIZATION_OF_RELATIONS': 2,
    'SCORING_FUNCTION_NORM': 1,
    'MARGIN_LOSS': 4,
    'LEARNING_RATE': 10,
    'NEGATIVE_SAMPLING_PROBS': [0.3, 0.0, 0.2, 0.5],
    'NEGATIVE_SAMPLING_TIMES': 10,
    'BATCH_SIZE': 8192,
    'EPOCHS': 100,
    'IS_QUINTS': False
}

In [ ]:
def sample_negatives(triple: List) -> List:
    if np.random.random() < 0.5:
        # sample subject
        return [random.choice(range(num_entities)), triple[1], triple[2]]
    else:
        # sample object
        return [triple[0], triple[1], random.choice(range(num_entities))]   

In [ ]:
def generate_negatives(positive: List[List], times: int):
    """
        :param postive: List of the raw data
        :param times: how many negative samples per positive sample.
    """
    negatives = []
    for pos in tqdm(positive):
        negatives_per_pos = [sample_negatives(pos) for _ in range(times)]
        negatives.append(negatives_per_pos)
        
    return negatives

In [ ]:
try:
    negatives = pickle.load(open(PRETRAINING_DATA_DIR / 'fb15k_negatives.pkl', 'rb'))
except (FileNotFoundError, IOError) as e:
    # Generate it again
    warnings.warn("Negative data not pre-generating. Takes three minutes.")
    negatives = generate_negatives(training_triples + valid_triples, 
                                   times = EXPERIMENT_CONFIG['NEGATIVE_SAMPLING_TIMES'])

    # Dump this somewhere
    with open(PRETRAINING_DATA_DIR / 'fb15k_negatives.pkl', 'wb+') as f:
        pickle.dump(negatives, f)

In [ ]:
train_neg = negatives[:len(training_triples)]
val_neg = negatives[len(training_triples):]

In [ ]:
data = {'train': {'pos': training_triples, 'neg': train_neg}, 'valid': {'pos': valid_triples, 'neg': val_neg}}

In [ ]:
training_triples[0:5], train_neg[0:5], valid_triples[0:5], val_neg[0:5]

## Model

In [ ]:
class BaseModule(nn.Module):
    """A base class for all of the models."""

    margin_ranking_loss_size_average: bool = None
    entity_embedding_max_norm: Optional[int] = None
    entity_embedding_norm_type: int = 2
    hyper_params = [EXPERIMENT_CONFIG['EMBEDDING_DIM'], 
                    EXPERIMENT_CONFIG['MARGIN_LOSS'], 
                    EXPERIMENT_CONFIG['LEARNING_RATE']]

    def __init__(self, config: Dict) -> None:
        super().__init__()

        # Device selection
        self.device = config['DEVICE']

        # Loss
        self.margin_loss = config['MARGIN_LOSS']
        self.criterion = nn.MarginRankingLoss(
            margin=self.margin_loss,
            reduction='mean' if self.margin_ranking_loss_size_average else 'sum'
        )

        # Entity dimensions
        #: The number of entities in the knowledge graph
        self.num_entities = config['NUM_ENTITIES']
        #: The number of unique relation types in the knowledge graph
        self.num_relations = config['NUM_RELATIONS']
        #: The dimension of the embeddings to generate
        self.embedding_dim = config['EMBEDDING_DIM']

        self.entity_embeddings = nn.Embedding(
            self.num_entities,
            self.embedding_dim,
            norm_type=self.entity_embedding_norm_type,
            max_norm=self.entity_embedding_max_norm,
        )

    def __init_subclass__(cls, **kwargs):  # noqa: D105
        if not getattr(cls, 'model_name', None):
            raise TypeError('missing model_name class attribute')

    def _get_entity_embeddings(self, entities):
        return self.entity_embeddings(entities).view(-1, self.embedding_dim)

    def _compute_loss(self, positive_scores: torch.Tensor, negative_scores: torch.Tensor) -> torch.Tensor:
        y = np.repeat([1], repeats=positive_scores.shape[0])
        y = torch.tensor(y, dtype=torch.float, device=self.device)

        loss = self.criterion(positive_scores, negative_scores, y)
        return loss

In [ ]:
def slice_triples(triples: torch.Tensor) -> List[torch.Tensor]:
    """ Slice in 3 or 5 as needed """
    return triples[:,0], triples[:,1], triples[:,2]



## Training

In [ ]:
config = EXPERIMENT_CONFIG.copy()
config['DEVICE'] = torch.device('cuda')
model = TransE(config)
model.to(config['DEVICE'])
optimizer = torch.optim.SGD(model.parameters(), lr=config['LEARNING_RATE'])


wandb.init(project="wikidata-embeddings")
for k, v in config.items():
    wandb.config[k] = v

In [ ]:
def evaluate_pointwise(pos_scores: torch.Tensor, neg_scores:torch.Tensor)->torch.Tensor:
    """
        Given a pos and neg quint, how many times did the score for positive be more than score for negative
    
        :param pos_scores: scores corresponding to pos quints (bs, )
        :param neg_scores: scores corresponding to neg quints (bs, )
        :return accuracy (0d tensor)
    """
    return torch.mean((pos_scores>=neg_scores).float()).item()
    
def evaluate_dataset(scores:torch.Tensor):
    """
        Compute score for `bs` set of [pos, neg, neg .....] quints.
        Assume pos is at the first position.
        
        
        :param scores: torch tensor of scores (bs,neg_samples+1)
        :returns (acc, mrr) both 1d tensors.
    """
    accuracy = (torch.argmax(scores, dim=1)==0).float()
    ranks = (torch.argsort(-scores, dim=1) == 0).nonzero()[:,1]
    print(ranks)
    recirank = 1.0/(ranks+1).float()
    
    return accuracy.detach().cpu().numpy(), recirank.detach().cpu().numpy()

In [ ]:
# Make a loop fn
def simplest_loop(epochs: int,
                  data: dict,
                  opt: torch.optim,
                  train_fn: Callable,
                  predict_fn: Callable,
                  device: torch.device = torch.device('cpu'),
                  data_fn: Callable = dataiters.SimplestSampler,
                  data_fn_val: Callable = dataiters.SimplestSampler,
                  eval_fn_trn: Callable = default_eval,
                  eval_fn_val: Callable = default_eval) -> (list, list, list):
    """
        A fn which can be used to train a language model.

        The model doesn't need to be an nn.Module,
            but have an eval (optional), a train and a predict function.

        Data should be a dict like so:
            {"train":{"x":np.arr, "y":np.arr}, "val":{"x":np.arr, "y":np.arr} }

        Train_fn must return both loss and y_pred

        :param epochs: number of epochs to train for
        :param data: a dict having keys train_x, test_x, train_y, test_y
        :param device: torch device to create new tensor from data
        :param opt: optimizer
        :param loss_fn: loss function
        :param train_fn: function to call with x and y
        :param predict_fn: function to call with x (test)
        :param data_fn: a class to which we can pass training data and get an iterator.
        :param data_fn_val: can be same as above; or diff. Specifically for validation runs.
        :param eval_fn: (optional) function which when given pred and true, returns acc
        :return: traces
    """

    train_loss = []
    train_acc = []
    valid_acc = []
    valid_acc_like_trn = []
    valid_mrr = []
    lrs = []

    # Epoch level
    for e in range(epochs):

        per_epoch_loss = []
        per_epoch_tr_acc = []

        # Train
        with Timer() as timer:

            # Make data
            trn_dl, val_dl_like_trn, val_dl = data_fn(data['train']), data_fn(data['valid']), data_fn_val(data['valid'])

            for pos, neg in tqdm(trn_dl):
                opt.zero_grad()

                _pos = torch.tensor(pos, dtype=torch.long, device=device)
                _neg = torch.tensor(neg, dtype=torch.long, device=device)

                (pos_scores, neg_scores), loss = train_fn(_pos, _neg)

                per_epoch_tr_acc.append(eval_fn_trn(pos_scores=pos_scores, neg_scores=neg_scores))
                per_epoch_loss.append(loss.item())

                loss.backward()
                opt.step()

        """
            # Val
            Run through the dataset twice.
                1. same as training data (pointwise eval)
                2. One quint (pos+negs) at a time. 
        """ 
        
        with torch.no_grad():

            per_epoch_vl_acc, per_epoch_vl_mrr, per_epoch_vl_acc_like_trn = [], [], []
            for quints in tqdm(val_dl):
                _quints = torch.tensor(quints, dtype=torch.long, device=device)
                
                # Flatten it
                _quints_shape = _quints.shape
                scores = predict_fn(_quints.view(-1, _quints_shape[-1]))
                scores = scores.view(_quints_shape[0], _quints_shape[1])
                
                accuracy, recirank = eval_fn_val(scores)

                per_epoch_vl_acc.append(accuracy)
                per_epoch_vl_mrr.append(recirank)
            
            for pos, neg in tqdm(val_dl_like_trn):
                
                _pos = torch.tensor(pos, dtype=torch.long, device=device)
                _neg = torch.tensor(neg, dtype=torch.long, device=device)

                (pos_scores, neg_scores), loss = train_fn(_pos, _neg)
                acc = eval_fn_trn(pos_scores=pos_scores, neg_scores=neg_scores)
                
                per_epoch_vl_acc_like_trn.append(acc)

        # Bookkeep
        train_acc.append(np.mean(per_epoch_tr_acc))
        train_loss.append(np.mean(per_epoch_loss))
        valid_acc.append(np.mean(per_epoch_vl_acc))
        valid_acc_like_trn.append(np.mean(per_epoch_vl_acc_like_trn))
        valid_mrr.append(np.mean(per_epoch_vl_mrr))
        
        print("Epoch: %(epo)03d | Loss: %(loss).5f | Tr_c: %(tracc)0.5f | Vl_c: %(vlacc)0.5f | Vl_c_liketrn: %(vlacc_liketrn)0.5f | Vl_mrr: %(vlmrr)0.5f |Time: %(time).3f min"
              % {'epo': e,
                 'loss': float(np.mean(per_epoch_loss)),
                 'tracc': float(np.mean(per_epoch_tr_acc)),
                 'vlacc': float(np.mean(per_epoch_vl_acc)),
                 'vlacc_liketrn': float(np.mean(per_epoch_vl_acc_like_trn)),
                 'vlmrr': float(np.mean(per_epoch_vl_mrr)),
                 'time': timer.interval / 60.0})

        # Wandb stuff
        wandb.log({
            'epoch': e, 
            'loss': float(np.mean(per_epoch_loss)),
            'trn_acc': float(np.mean(per_epoch_tr_acc)),
            'val_acc': float(np.mean(per_epoch_vl_acc)),
            'val_acc_liketrn': float(np.mean(per_epoch_vl_acc_like_trn)),
            'val_mrr': float(np.mean(per_epoch_vl_mrr))
        })

    return train_acc, valid_acc, valid_acc_like_trn, valid_mrr, train_loss

In [ ]:
args = {
    "epochs":config['EPOCHS'],
    "data":data,
    "opt": optimizer,
    "train_fn": model,
    "predict_fn": model.predict,
    "device": config['DEVICE'],
    "data_fn": partial(QuintRankingSampler, bs=config["BATCH_SIZE"]),
    "data_fn_val": partial(SingleSampler, bs=config["BATCH_SIZE"]),
    "eval_fn_trn": evaluate_pointwise,
    "eval_fn_val": evaluate_dataset
}

In [ ]:
simplest_loop(**args)